In [1]:
from keras.datasets import boston_housing
(train_data, train_targets), (test_data, test_targets) = boston_housing.load_data()

Using TensorFlow backend.


In [2]:
train_data.shape

(404, 13)

In [3]:
test_data.shape

(102, 13)

In [4]:
train_targets

array([15.2, 42.3, 50. , 21.1, 17.7, 18.5, 11.3, 15.6, 15.6, 14.4, 12.1,
       17.9, 23.1, 19.9, 15.7,  8.8, 50. , 22.5, 24.1, 27.5, 10.9, 30.8,
       32.9, 24. , 18.5, 13.3, 22.9, 34.7, 16.6, 17.5, 22.3, 16.1, 14.9,
       23.1, 34.9, 25. , 13.9, 13.1, 20.4, 20. , 15.2, 24.7, 22.2, 16.7,
       12.7, 15.6, 18.4, 21. , 30.1, 15.1, 18.7,  9.6, 31.5, 24.8, 19.1,
       22. , 14.5, 11. , 32. , 29.4, 20.3, 24.4, 14.6, 19.5, 14.1, 14.3,
       15.6, 10.5,  6.3, 19.3, 19.3, 13.4, 36.4, 17.8, 13.5, 16.5,  8.3,
       14.3, 16. , 13.4, 28.6, 43.5, 20.2, 22. , 23. , 20.7, 12.5, 48.5,
       14.6, 13.4, 23.7, 50. , 21.7, 39.8, 38.7, 22.2, 34.9, 22.5, 31.1,
       28.7, 46. , 41.7, 21. , 26.6, 15. , 24.4, 13.3, 21.2, 11.7, 21.7,
       19.4, 50. , 22.8, 19.7, 24.7, 36.2, 14.2, 18.9, 18.3, 20.6, 24.6,
       18.2,  8.7, 44. , 10.4, 13.2, 21.2, 37. , 30.7, 22.9, 20. , 19.3,
       31.7, 32. , 23.1, 18.8, 10.9, 50. , 19.6,  5. , 14.4, 19.8, 13.8,
       19.6, 23.9, 24.5, 25. , 19.9, 17.2, 24.6, 13

# Preparing the data
'''It would be problematic to feed into a neural network values that all take wildly different
ranges. The network might be able to automatically adapt to such heterogeneous
data, but it would definitely make learning more difficult.A widespread best practice
to deal with such data is to do feature-wise normalization: for each feature in the input
data (a column in the input data matrix), you subtract the mean of the feature and
divide by the standard deviation, so that the feature is centered around 0 and has a
unit standard deviation. This is easily done in Numpy'''


In [5]:
# this step is of rescaling
mean = train_data.mean(axis=0)
train_data -= mean
std = train_data.std(axis=0)
train_data /= std
test_data -= mean
test_data /= std

You should never use in your workflow any quantity computed on the
test data


### **Building your network**
Because so few samples are available, you’ll use a very small network with two hidden
layers, each with 64 units. In general, the less training data you have, the worse overfitting
will be, and using a small network is one way to mitigate overfitting.

In [6]:
#Listing 3.26 Model definition

In [14]:
from keras import models
from keras import layers
def build_model():
    
    model = models.Sequential()
    model.add(layers.Dense(64, activation='relu',input_shape=(train_data.shape[1],)))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(1)) # regration main is wajah se koi activation nhi use kiya kyu ke ye 1 value deta hai. matlab 1 hi output
    model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])
    return model

The network ends with a single unit and no activation (it will be a linear layer). This is
a typical setup for scalar regression (a regression where you’re trying to predict a single
continuous value). Applying an activation function would constrain the range the output
can take; for instance, if you applied a sigmoid activation function to the last layer,
the network could only learn to predict values between 0 and 1. Here, because the last
layer is purely linear, the network is free to learn to predict values in any range.

Note that you compile the network with the mse loss function—mean squared error,
the square of the difference between the predictions and the targets. This is a widely
used loss function for regression problems.
You’re also monitoring a new metric during training: mean absolute error (MAE). It’s
the absolute value of the difference between the predictions and the targets. For
instance, an MAE of 0.5 on this problem would mean your predictions are off by $500
on average.

### **Validating your approach using K-fold validation**
To evaluate your network while you keep adjusting its parameters (such as the number
of epochs used for training), you could split the data into a training set and a validation
set, as you did in the previous examples. But because you have so few data points,
the validation set would end up being very small (for instance, about 100 examples).
As a consequence, the validation scores might change a lot depending on which data
points you chose to use for validation and which you chose for training: the validation
scores might have a high variance with regard to the validation split. This would prevent
you from reliably evaluating your model.
The best practice in such situations is to use K-fold cross-validation (see figure 3.11).
It consists of splitting the available data into K partitions (typically K = 4 or 5), instantiating
K identical models, and training each one on K – 1 partitions while evaluating on
the remaining partition. The validation score for the model used is then the average of
the K validation scores obtained. In terms of code, this is straightforward.

In [15]:
# Listing 3.27 K-fold validation
#page no 88 

In [16]:
import numpy as np
k = 4
num_val_samples = len(train_data) // k
num_epochs = 100
all_scores = []

In [17]:
#Prepares the validation data: data from partition #k
for i in range(k):
    
    print('processing fold #', i)
    val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
    
    val_targets = train_targets[i * num_val_samples: (i + 1) * num_val_samples]
    #Prepares the training data: data from all other partitions
    partial_train_data = np.concatenate([train_data[:i * num_val_samples],train_data[(i + 1) * num_val_samples:]],axis=0)
    partial_train_targets = np.concatenate([train_targets[:i * num_val_samples],train_targets[(i + 1) * num_val_samples:]],axis=0)
    
    # Builds the Keras model (already compiled)
    model = build_model()
    #Trains the model (in silent mode,verbose = 0)
    model.fit(partial_train_data, partial_train_targets,epochs=num_epochs, batch_size=1, verbose=1)
    # Evaluates the model on the validation data
    val_mse, val_mae = model.evaluate(val_data, val_targets, verbose=1)
    all_scores.append(val_mae)

processing fold # 0






Epoch 1/100
303/303 [==============================] - 4s 14ms/step - loss: 202.3724 - mean_absolute_error: 10.7718
Epoch 2/100
303/303 [==============================] - 0s 2ms/step - loss: 30.2423 - mean_absolute_error: 3.8338
Epoch 3/100
303/303 [==============================] - 0s 1ms/step - loss: 21.3060 - mean_absolute_error: 3.1748
Epoch 4/100
303/303 [==============================] - 0s 1ms/step - loss: 18.2513 - mean_absolute_error: 2.8964
Epoch 5/100
303/303 [==============================] - 0s 2ms/step - loss: 16.6121 - mean_absolute_error: 2.6646
Epoch 6/100
303/303 [==============================] - 0s 1ms/step - loss: 15.4777 - mean_absolute_error: 2.5688
Epoch 7/100
303/303 [==============================] - 0s 2ms/step - loss: 14.2278 - mean_absolute_error: 2.5345
Epoch 8/100
303/303 [==============================] - 0s 1ms/step - loss: 13.8438 - mean_absolute_error: 2.4211
Epoch 9/100
303/303 [==============================] - 0s 1ms/step 

303/303 [==============================] - 1s 2ms/step - loss: 6.0658 - mean_absolute_error: 1.5872
Epoch 62/100
303/303 [==============================] - 1s 2ms/step - loss: 5.9950 - mean_absolute_error: 1.5142
Epoch 63/100
303/303 [==============================] - 0s 2ms/step - loss: 5.3623 - mean_absolute_error: 1.5708
Epoch 64/100
303/303 [==============================] - 0s 2ms/step - loss: 5.8061 - mean_absolute_error: 1.5779
Epoch 65/100
303/303 [==============================] - 0s 2ms/step - loss: 5.7416 - mean_absolute_error: 1.5932
Epoch 66/100
303/303 [==============================] - 0s 1ms/step - loss: 5.5920 - mean_absolute_error: 1.5683
Epoch 67/100
303/303 [==============================] - 0s 2ms/step - loss: 5.4523 - mean_absolute_error: 1.5082
Epoch 68/100
303/303 [==============================] - 0s 2ms/step - loss: 5.1539 - mean_absolute_error: 1.4739
Epoch 69/100
303/303 [==============================] - 0s 2ms/step - loss: 5.5082 - mean_absolute_error: 1.4

303/303 [==============================] - 1s 2ms/step - loss: 8.0288 - mean_absolute_error: 1.8838
Epoch 33/100
303/303 [==============================] - 0s 2ms/step - loss: 7.6608 - mean_absolute_error: 1.8311
Epoch 34/100
303/303 [==============================] - 1s 2ms/step - loss: 8.0782 - mean_absolute_error: 1.8491
Epoch 35/100
303/303 [==============================] - 1s 2ms/step - loss: 8.0246 - mean_absolute_error: 1.8012
Epoch 36/100
303/303 [==============================] - 0s 2ms/step - loss: 7.3747 - mean_absolute_error: 1.7543
Epoch 37/100
303/303 [==============================] - 0s 2ms/step - loss: 7.9022 - mean_absolute_error: 1.8006
Epoch 38/100
303/303 [==============================] - 0s 2ms/step - loss: 8.0655 - mean_absolute_error: 1.8040
Epoch 39/100
303/303 [==============================] - 1s 2ms/step - loss: 7.2271 - mean_absolute_error: 1.7286
Epoch 40/100
303/303 [==============================] - 0s 2ms/step - loss: 7.4834 - mean_absolute_error: 1.8

303/303 [==============================] - 0s 2ms/step - loss: 19.8273 - mean_absolute_error: 3.1652
Epoch 4/100
303/303 [==============================] - 0s 1ms/step - loss: 16.2112 - mean_absolute_error: 2.7828
Epoch 5/100
303/303 [==============================] - 0s 1ms/step - loss: 14.0475 - mean_absolute_error: 2.6260
Epoch 6/100
303/303 [==============================] - 1s 2ms/step - loss: 12.4989 - mean_absolute_error: 2.4977
Epoch 7/100
303/303 [==============================] - 0s 2ms/step - loss: 11.3247 - mean_absolute_error: 2.4349
Epoch 8/100
303/303 [==============================] - 1s 2ms/step - loss: 10.2658 - mean_absolute_error: 2.2858
Epoch 9/100
303/303 [==============================] - 0s 2ms/step - loss: 10.7469 - mean_absolute_error: 2.2386
Epoch 10/100
303/303 [==============================] - 0s 2ms/step - loss: 9.7329 - mean_absolute_error: 2.2218
Epoch 11/100
303/303 [==============================] - 0s 2ms/step - loss: 9.5399 - mean_absolute_error: 2.

303/303 [==============================] - 1s 2ms/step - loss: 3.8244 - mean_absolute_error: 1.3751
Epoch 76/100
303/303 [==============================] - 1s 2ms/step - loss: 3.9938 - mean_absolute_error: 1.3809
Epoch 77/100
303/303 [==============================] - 1s 2ms/step - loss: 3.8899 - mean_absolute_error: 1.3554
Epoch 78/100
303/303 [==============================] - 1s 2ms/step - loss: 3.4944 - mean_absolute_error: 1.3616
Epoch 79/100
303/303 [==============================] - 1s 2ms/step - loss: 3.4249 - mean_absolute_error: 1.3940
Epoch 80/100
303/303 [==============================] - 1s 2ms/step - loss: 3.6939 - mean_absolute_error: 1.3848
Epoch 81/100
303/303 [==============================] - 1s 2ms/step - loss: 3.7396 - mean_absolute_error: 1.3662
Epoch 82/100
303/303 [==============================] - 1s 2ms/step - loss: 3.7422 - mean_absolute_error: 1.3382
Epoch 83/100
303/303 [==============================] - 1s 2ms/step - loss: 3.5622 - mean_absolute_error: 1.3

303/303 [==============================] - 1s 2ms/step - loss: 6.6658 - mean_absolute_error: 1.7146
Epoch 47/100
303/303 [==============================] - 1s 2ms/step - loss: 6.9852 - mean_absolute_error: 1.6739
Epoch 48/100
303/303 [==============================] - 1s 2ms/step - loss: 6.7136 - mean_absolute_error: 1.6380
Epoch 49/100
303/303 [==============================] - 1s 2ms/step - loss: 6.6595 - mean_absolute_error: 1.6708
Epoch 50/100
303/303 [==============================] - 1s 2ms/step - loss: 6.5305 - mean_absolute_error: 1.6274
Epoch 51/100
303/303 [==============================] - 1s 2ms/step - loss: 6.4829 - mean_absolute_error: 1.6190
Epoch 52/100
303/303 [==============================] - 1s 2ms/step - loss: 6.4774 - mean_absolute_error: 1.6025
Epoch 53/100
303/303 [==============================] - 1s 2ms/step - loss: 5.9088 - mean_absolute_error: 1.5974
Epoch 54/100
303/303 [==============================] - 1s 2ms/step - loss: 6.6233 - mean_absolute_error: 1.6

In [18]:
all_scores

[2.020513403533709, 2.243280361194422, 2.8713827912170107, 2.42396471819075]